In [1]:
# import
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.embeddings.openai import OpenAIEmbedding
from IPython.display import Markdown, display
import chromadb
import os
import getpass

import openai

openai.api_key = os.getenv("OPENAI_API_KEY")
# create client and a new collection
chroma_client = chromadb.EphemeralClient()
chroma_collection = chroma_client.create_collection("quickstart")

In [2]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"Llama-index FYP"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv('LANGCHAIN_API_KEY')

In [13]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_extraction_chain
from dataclasses import dataclass
chroma_directory = 'db/'

db = Chroma(persist_directory=chroma_directory, embedding_function=OpenAIEmbeddings())
#file_path = "JiaHao_Lo.pdf"
#file_path = "GooYeJui_CV.pdf"
file_path = "resume/Ang Teik Hun Resume.pdf"
loader = PyPDFLoader(file_path)
documents = loader.load()


@dataclass
class Document:
    page_content: str
    metadata: dict

def extract_information(documents):
    # Schema
    schema = {
        "properties": {
            "name": {"type": "string"},
            "phone_number": {"type": "string"},
            "email": {"type": "string"},
            "local": {"type": "string"},
            "last role": {"type": "string"},
            "years of experience": {"type": "string"},
            "education level": {"type": "string"},
            "CGPA": {"type": "integer"},
            "University": {"type": "string"},
            "Education Background": {"type": "string"},
            "Data Science Background": {"type": "string"},
            "Relevant experience": {"type": "string"},
        },
        "required": ["name", "height"],
    }

    # Input
    inp = documents

    # Run chain
    llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125",verbose=True)
    chain = create_extraction_chain(schema, llm,verbose=True)
    result = chain.run(inp)
    return result

results = extract_information(documents)

document_objects = []
# Construct page content from dictionary values
page_content = ""
for key, value in results[0].items():
    page_content += f"{key}: {value}\n"

# Construct metadata
metadata = {'source': file_path, 'page': 0}  # You may adjust the source accordingly

# Create Document object
document = Document(page_content=page_content, metadata=metadata)
document_objects.append(document)

print(document_objects)
db.add_documents(documents=document_objects)

/workspaces/fyp_goo/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/workspaces/fyp_goo/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/workspaces/fyp_goo/venv



> Entering new LLMChain chain...
Prompt after formatting:
Human: Extract and save the relevant entities mentioned in the following passage together with their properties.

Only extract the properties mentioned in the 'information_extraction' function.

If a property is not present and is not required in the function parameters, do not include it in the output.

Passage:
[Document(page_content=' \nCurriculum Vitae/Resume  \nName: Ang Teik Hun  \n \nPROFILE  \nA highly motivated and skilled data scientist with a strong background in computer science and statistical analysis. \nProven experience in using a variety of data mining and machine learning techniques to solve complex problems. \nProficient in Python, R, and  SQL.  \n \n EXPERIENCE  \nInternship as Data Scientist, Petroliam Nasional Berhad  \nGroup Digital , KL / November 2020  – July 2021  \n• Used regular expression and other text pre -processing features in python \nto automate the extraction of information  from corporate b

['addf84f2-d5fd-11ee-9f57-000d3ac8563d']

In [5]:
# create the pack
job_description = """Responsible for design, planning, and coordinating the implementation of Data Science work activities in the Group Digital with established structured processes and procedures to support PETRONAS's digital agenda.
 

1) Technical & Professional Excellence

Responsible for ensuring data required for analytic models is of required quality and models are constructed to standards and deployed effectively.
Implement data science industry best practices, relevant cutting-edge technology, and innovation in projects to ensure the right solutions fulfill the business requirements.

2) Technical/Skill Leadership & Solutioning

Responsible for developing appropriate technical solutions to address business pain points with insights and recommendations.
Implement an established analytics strategy by adopting the right technologies and technical requirements when executing projects to ensure business value generation.
Execute operational excellence through continuous technical and process improvement initiatives within projects to improve operations efficiency and effectiveness within own activity & projects.

3) Technical Expertise

Track and follow up with relevant parties to ensure Technical Excellence Programmes are successfully deployed and integrated into work processes, documents, policies, and guidelines.
Participate in a community of practices and network with internal and external technical experts by identifying solutions to common problems and capturing and sharing existing data science knowledge for continuous excellence.

 

Be part of our DS team in at least one of the following areas:

Machine Learning

Roles: Design analytics solutions for business problems; develop, evaluate, optimize, deploy and maintain models.

Tech stack: ML Algorithms, Python, SQL, Spark, Git, Cloud Services, Deep Learning frameworks, MLOps, etc

 

Natural Language Processing

Roles: Design text analytics solutions for business problems; develop, evaluate, optimize, deploy and maintain text processing and analytics solutions.

Tech stack: Python, SQL, Git, NLTK, Deep Learning frameworks, MLOps, Text analytics, NLP, NLU, NLG, Language Models, etc

 

Computer Vision

Roles: Design Image and video analytics solutions for business problems; develop, evaluate, optimize, deploy and maintain solutions

Tech stack: Tensorflow, OpenCV, Fastai, Pytorch, MLFlow, Spark, MLlib Python, SQL, Git, Deep Learning frameworks, MLOps, etc

 

Optimization / Simulation

Roles: Design optimization/simulation analytics solutions for business problems; develop, evaluate, optimize, deploy and maintain solutions

Tech stack: mathematical/process models, Simulation modeling, AnyLogic, Simio, mixed-integer programming (linear and nonlinear), Python, Pyomo, Gurobi solver, MLOps, etc."""

job_requirements = """
BS or MS in Data Science 5 to 15 years of experience in electronic package design/ mechanical enclosure design for hand held products.
"""

In [4]:
job_description="""
Develop, design, and implement mechanical solutions including but not limited to enclosures and mechanisms while managing constraints in space, time, cost, manufacturing, structural integrity, and thermal loads.

Direct and coordinate mechanical engineering and manufacturing services as required.

Develop project documentation including specifications, 3D CAD models, 2D drawings, schedules and presentations to capture and communicate the mechanical design to achieve the defined project goals.
"""

job_requirements = """
BS or MS in Mechanical Engineering with 5 to 15 years of experience in electronic package design/ mechanical enclosure design for hand held products.
"""

In [35]:
from pathlib import Path
from typing import Any, Dict, List, Optional
from llama_index.core import ServiceContext
from llama_index.core.llama_pack.base import BaseLlamaPack
from llama_index.core.response_synthesizers import TreeSummarize
from llama_index.core.schema import NodeWithScore
from llama_index.llms.openai import OpenAI
from llama_index.readers.file import PDFReader
from pydantic import BaseModel, Field
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_extraction_chain
from dataclasses import dataclass



@dataclass
class Document:
    page_content: str
    metadata: dict



# backwards compatibility
try:
    from llama_index.core.llms.llm import LLM
except ImportError:
    from llama_index.core.llms.base import LLM

QUERY_TEMPLATE = """
[Instruction] You will be provided with details such as the screening criteria, job description and job requirements.
Please act as an impartial judge and evaluate the candidate's {criteria_str} based on the job description and job requirements. For this evaluation, you should primarily consider the following accuracy:
[Accuracy]
Score 1: The candidate's {criteria_str} is completely unrelated to job_description and job_requirements.
Score 3: The candidate's {criteria_str} has minor relevance but does not align with job_description and job_requirements.
Score 5: The candidate's {criteria_str} has moderate relevance but contains inaccuracies to job_description and job_requirements.
Score 7: The candidate's {criteria_str} aligns with job_description and job_requirements. but has minor errors or omissions on either one of them.
Score 10: The candidate's {criteria_str} is completely accurate and aligns very well with job_description and job_requirements.
        

### Job Description
{job_description}

### Job Requirements
{job_requirements}

### Screening Criteria
{criteria_str}
"""


class CriteriaDecision(BaseModel):
    """The decision made based on a single criteria."""

    scoring: int = Field(description="The scoring in the scale of 1-10 made based on the criteria")
    reasoning: str = Field(description="The reasoning behind the decision")


class ResumeScreenerDecision(BaseModel):
    """The decision made by the resume screener."""

    criteria_decisions: List[CriteriaDecision] = Field(
        description="The decisions made based on the criteria"
    )
    overall_reasoning: str = Field(
        description="The reasoning behind the overall decision"
    )
    scoring: int = Field(
        description="The overall scoring in the scale of 1-10 made based on the overall criteria"
    )


def _format_criteria_str(criteria: List[str]) -> str:
    criteria_str = ""
    for criterion in criteria:
        criteria_str += f"- {criterion}\n"
    return criteria_str


class ResumeScreenerPack(BaseLlamaPack):
    def __init__(
        self, job_description: str, job_requirements:str, criteria: str, llm: Optional[LLM] = None
    ) -> None:
        self.reader = PDFReader()
        llm = llm or OpenAI(model="gpt-3.5-turbo-0125")
        service_context = ServiceContext.from_defaults(llm=llm)
        self.synthesizer = TreeSummarize(
            output_cls=ResumeScreenerDecision, service_context=service_context
        )
        self.query = QUERY_TEMPLATE.format(
            job_description=job_description, job_requirements=job_requirements, criteria_str=criteria
        )

    def get_modules(self) -> Dict[str, Any]:
        """Get modules."""
        return {"reader": self.reader, "synthesizer": self.synthesizer}

    def run(self, resume_path: str, *args: Any, **kwargs: Any) -> Any:
        """Run pack."""
        docs = self.reader.load_data(Path(resume_path))
        print(docs)
        for doc in docs:
            print(type(doc))
            print("printing doc",doc)

        output = self.synthesizer.synthesize(
            query=self.query,
            nodes=[NodeWithScore(node=doc, score=1.0) for doc in docs],
        )
        return output.response
    
criteria_list=["educational background"]
results = []

for criteria in criteria_list:
    resume_screener = ResumeScreenerPack(
        job_description=job_description,
        job_requirements=job_requirements,
        criteria=criteria,
    )
    response = resume_screener.run(resume_path="resume/Ang Teik Hun Resume.pdf")
    print(response)
    results.append(response)

C:\Users\user\AppData\Local\Temp\ipykernel_9200\104807160.py:90: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=llm)


[Document(id_='19c0c00a-1f29-44ec-8174-e5874bd75b60', embedding=None, metadata={'page_label': '1', 'file_name': 'c:/Users/user/Desktop/fyp_goo/trash/resume/Ang Teik Hun Resume.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text=' \nCurriculum Vitae/Resume  \nName: Ang Teik Hun  \n \nPROFILE  \nA highly motivated and skilled data scientist with a strong background in computer science and statistical analysis. \nProven experience in using a variety of data mining and machine learning techniques to solve complex problems. \nProficient in Python, R, and  SQL.  \n \n EXPERIENCE  \nInternship as Data Scientist, Petroliam Nasional Berhad  \nGroup Digital , KL / November 2020  – July 2021  \n• Used regular expression and other text pre -processing features in python \nto automate the extraction of information  from corporate business \ndatabase , minimizing data quality issues by 90% . \n• Utilized Object Orient ed Programming (class and subclass) for 

In [38]:
from llama_index.core import ServiceContext
from llama_index.core.llama_pack.base import BaseLlamaPack
from llama_index.core.response_synthesizers import TreeSummarize
from llama_index.core.schema import NodeWithScore
from llama_index.llms.openai import OpenAI
from pydantic import BaseModel, Field
from typing import Optional, Dict, Any, List

# backwards compatibility
try:
    from llama_index.core.llms.llm import LLM
except ImportError:
    from llama_index.core.llms.base import LLM

QUERY_TEMPLATE = """
[Instruction] You will be provided with details such as the screening criteria, job description and job requirements.
Please act as an impartial judge and evaluate the candidate's {criteria_str} based on the job description and job requirements. For this evaluation, you should primarily consider the following accuracy:
[Accuracy]
Score 1: The candidate's {criteria_str} is completely unrelated to job_description and job_requirements.
Score 3: The candidate's {criteria_str} has minor relevance but does not align with job_description and job_requirements.
Score 5: The candidate's {criteria_str} has moderate relevance but contains inaccuracies to job_description and job_requirements.
Score 7: The candidate's {criteria_str} aligns with job_description and job_requirements. but has minor errors or omissions on either one of them.
Score 10: The candidate's {criteria_str} is completely accurate and aligns very well with job_description and job_requirements.
        

### Job Description
{job_description}

### Job Requirements
{job_requirements}

### Screening Criteria
{criteria_str}
"""


class CriteriaDecision(BaseModel):
    """The decision made based on a single criteria."""

    scoring: int = Field(description="The scoring in the scale of 1-10 made based on the criteria")
    reasoning: str = Field(description="The reasoning behind the decision")


class ResumeScreenerDecision(BaseModel):
    """The decision made by the resume screener."""

    criteria_decisions: List[CriteriaDecision] = Field(
        description="The decisions made based on the criteria"
    )
    overall_reasoning: str = Field(
        description="The reasoning behind the overall decision"
    )
    scoring: int = Field(
        description="The overall scoring in the scale of 1-10 made based on the overall criteria"
    )


def _format_criteria_str(criteria: List[str]) -> str:
    criteria_str = ""
    for criterion in criteria:
        criteria_str += f"- {criterion}\n"
    return criteria_str


class ResumeScreenerPack(BaseLlamaPack):
    def __init__(
        self, job_description: str, job_requirements:str, criteria: str, llm: Optional[LLM] = None
    ) -> None:
        self.reader = extract_information
        llm = llm or OpenAI(model="gpt-4")
        service_context = ServiceContext.from_defaults(llm=llm)
        self.synthesizer = TreeSummarize(
            output_cls=ResumeScreenerDecision, service_context=service_context
        )
        self.query = QUERY_TEMPLATE.format(
            job_description=job_description, job_requirements=job_requirements, criteria_str=criteria
        )

    def get_modules(self) -> Dict[str, Any]:
        """Get modules."""
        return {"reader": self.reader, "synthesizer": self.synthesizer}

    def run(self, resume_path: str, *args: Any, **kwargs: Any) -> Any:
        """Run pack."""
        docs = self.reader(resume_path)
        print(docs)
        for doc in docs:
            print("printing doc",doc)

        output = self.synthesizer.synthesize(
            query=self.query,
            nodes=[NodeWithScore(node=doc, score=1.0) for doc in docs],
        )
        return output.response
    
criteria_list=["educational background", "skill groups","job experience"]
results = []

for criteria in criteria_list:
    resume_screener = ResumeScreenerPack(
        job_description=job_description,
        job_requirements=job_requirements,
        criteria=criteria,
    )
    response = resume_screener.run(resume_path="resume/JiaHao_Lo.pdf")
    print(response)
    results.append(response)

C:\Users\user\AppData\Local\Temp\ipykernel_9200\3797911861.py:81: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=llm)




> Entering new LLMChain chain...
Prompt after formatting:
Human: Extract and save the relevant entities mentioned in the following passage together with their properties.

Only extract the properties mentioned in the 'information_extraction' function.

If a property is not present and is not required in the function parameters, do not include it in the output.

Passage:
[Document(page_content='Jia-Hao Lo   \njiahaolo@u.northwestern.edu | www.linkedin.com/in/jiahaolo | (872) -810-5588  \nEDUCATION \n \nNorthwestern University CGPA: 3. 83/4.00                                                                                        Evanston, Illinois  \nMaster of Science in Analytics                                                                                     Sept 2021 – Dec 202 2 (Expected)  \nPETRONAS Educational Sponsorship Program 2015 ( Full sponsorship for postgraduate study) \n \nUniversity Technology P ETRONAS  CGPA: 3.92/4. 00                                               

C:\Users\user\AppData\Local\Temp\ipykernel_9200\3797911861.py:81: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=llm)



> Finished chain.
Doc ID: f9761daf-dacf-45d4-82e3-3e511bd83779
Text: [{'name': 'Jia-Hao Lo', 'phone_number': '(872) -810-5588',
'email': 'jiahaolo@u.northwestern.edu', 'local': 'Evanston, Illinois',
'education level': 'Master of Science in Analytics', 'CGPA': 383,
'University': 'Northwestern University', 'Education Background':
'Bachelor of Civil Engineering in Offshore Engineering', 'Relevant
experience': 'Data ...
[Document(id_='f9761daf-dacf-45d4-82e3-3e511bd83779', embedding=None, metadata={'page_label': '1', 'file_name': 'resume/JiaHao_Lo.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="[{'name': 'Jia-Hao Lo', 'phone_number': '(872) -810-5588', 'email': 'jiahaolo@u.northwestern.edu', 'local': 'Evanston, Illinois', 'education level': 'Master of Science in Analytics', 'CGPA': 383, 'University': 'Northwestern University', 'Education Background': 'Bachelor of Civil Engineering in Offshore Engineering', 'Relevant experience': 'Data Science

C:\Users\user\AppData\Local\Temp\ipykernel_9200\3797911861.py:81: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=llm)



> Finished chain.
Doc ID: 6ba93539-a2a0-4a05-8ab3-11f755e3a67f
Text: [{'name': 'Jia-Hao Lo', 'phone_number': '(872) -810-5588',
'email': 'jiahaolo@u.northwestern.edu', 'local': 'Evanston, Illinois',
'education level': 'Master of Science in Analytics', 'CGPA': 3.83,
'University': 'Northwestern University', 'Education Background':
'Bachelor of Civil Engineering in Offshore Engineering', 'Data Science
Background': '...
[Document(id_='6ba93539-a2a0-4a05-8ab3-11f755e3a67f', embedding=None, metadata={'page_label': '1', 'file_name': 'resume/JiaHao_Lo.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="[{'name': 'Jia-Hao Lo', 'phone_number': '(872) -810-5588', 'email': 'jiahaolo@u.northwestern.edu', 'local': 'Evanston, Illinois', 'education level': 'Master of Science in Analytics', 'CGPA': 3.83, 'University': 'Northwestern University', 'Education Background': 'Bachelor of Civil Engineering in Offshore Engineering', 'Data Science Background': 'Data Sc

In [39]:
print(response)

criteria_decisions=[CriteriaDecision(scoring=1, reasoning="The candidate's job experience is completely unrelated to the job description and job requirements. The candidate has a background in data science and analytics, with experience in areas such as exploratory data analysis, hypothesis testing, clustering analysis, and machine learning. However, the job description and requirements are focused on mechanical engineering and manufacturing, specifically in the area of electronic package design/mechanical enclosure design for hand-held products. The candidate does not have any experience or education in this area.")] overall_reasoning="The candidate's job experience and educational background do not align with the job description and job requirements. The candidate's experience is in data science and analytics, while the job is in mechanical engineering and manufacturing." scoring=1


In [41]:
response = resume_screener.run(resume_path="resume/Ang Teik Hun Resume.pdf")
print(response)

[Document(id_='2536a754-4ce1-4a78-ae14-3ae5a944f6f0', embedding=None, metadata={'page_label': '1', 'file_name': '/workspaces/fyp_goo/trash/resume/Ang Teik Hun Resume.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text=' \nCurriculum Vitae/Resume  \nName: Ang Teik Hun  \n \nPROFILE  \nA highly motivated and skilled data scientist with a strong background in computer science and statistical analysis. \nProven experience in using a variety of data mining and machine learning techniques to solve complex problems. \nProficient in Python, R, and  SQL.  \n \n EXPERIENCE  \nInternship as Data Scientist, Petroliam Nasional Berhad  \nGroup Digital , KL / November 2020  – July 2021  \n• Used regular expression and other text pre -processing features in python \nto automate the extraction of information  from corporate business \ndatabase , minimizing data quality issues by 90% . \n• Utilized Object Orient ed Programming (class and subclass) for the \nconv

NameError: name 'criteria' is not defined

In [16]:
response = resume_screener.run(resume_path="resume/GooYeJui_CV.pdf")
print(response)

[Document(id_='38c156bb-ce4a-422c-b173-bffb89325088', embedding=None, metadata={'page_label': '1', 'file_name': '/workspaces/fyp_goo/trash/resume/GooYeJui_CV.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="SEPT 2023 - JUNE 2024\nJUNE 2021 - JULY 2021 PRODUCTION OPERATOR\nJINKO SOLAR TECHNOLOGY SDN BHDDATA SCIENCE INTERN\nPETRONAS DIGITAL SDN. BHD.\nAssisted in the development of machine learning model contributing to the automation of critical business processes.\nCollaborated with a team of data professionals to work on live projects, applying data analytics and machine learning\nto derive actionable insights and drive innovation. Gained hands-on experience in data science methodologies, tools,\nand technologies, furthering the journey toward becoming a proficient data scientist.\nApplied techniques like natural language processing and OpenAI GPT-3 text-davinci-003 engine on the Resume\nParser App.\nIntegrated a new feature to automate th

In [36]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_extraction_chain
from llama_index.core.schema import Document
import uuid

def generate_random_id():
    random_id = uuid.uuid4()
    return str(random_id)

def extract_information(file_path):
    # Schema
    schema = {
        "properties": {
            "name": {"type": "string"},
            "phone_number": {"type": "string"},
            "email": {"type": "string"},
            "local": {"type": "string"},
            "last role": {"type": "string"},
            "years of experience": {"type": "string"},
            "education level": {"type": "string"},
            "CGPA": {"type": "integer"},
            "University": {"type": "string"},
            "Education Background": {"type": "string"},
            "Data Science Background": {"type": "string"},
            "Relevant experience": {"type": "string"},
        },
        "required": ["name", "height"],
    }

    loader = PyPDFLoader(file_path)
    documents = loader.load()
    document_objects = []

    # Run chain
    llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125",verbose=True)
    chain = create_extraction_chain(schema, llm,verbose=True)
    result = chain.run(documents)
    metadata = {'page_label': '1', 'file_name': file_path}  


    # Create Document object
    document = Document(metadata=metadata,text=str(result))

    print(document)
    document_objects.append(document)
    return document_objects

results_ex = extract_information("resume/JiaHao_Lo.pdf")



> Entering new LLMChain chain...
Prompt after formatting:
Human: Extract and save the relevant entities mentioned in the following passage together with their properties.

Only extract the properties mentioned in the 'information_extraction' function.

If a property is not present and is not required in the function parameters, do not include it in the output.

Passage:
[Document(page_content='Jia-Hao Lo   \njiahaolo@u.northwestern.edu | www.linkedin.com/in/jiahaolo | (872) -810-5588  \nEDUCATION \n \nNorthwestern University CGPA: 3. 83/4.00                                                                                        Evanston, Illinois  \nMaster of Science in Analytics                                                                                     Sept 2021 – Dec 202 2 (Expected)  \nPETRONAS Educational Sponsorship Program 2015 ( Full sponsorship for postgraduate study) \n \nUniversity Technology P ETRONAS  CGPA: 3.92/4. 00                                               

In [12]:
results_ex

[Document(id_='99916367-63c6-4a45-bdad-1ab62593b1ba', embedding=None, metadata={'page_label': '1', 'file_name': 'resume/JiaHao_Lo.pdf'}, text='name: Jia-Hao Lo\nphone_number: (872) -810-5588\nemail: jiahaolo@u.northwestern.edu\nlocal: Evanston, Illinois\neducation level: Master of Science in Analytics\nCGPA: 383\nUniversity: Northwestern University\nEducation Background: Bachelor of Civil Engineering in Offshore Engineering\nRelevant experience: Data Science Department On-Job Training, Engineering Internship\nyears of experience: Nov 2020 – July 2021, May 2019 – Dec 2019\n')]

In [26]:
dict = {'name': 'Jia-Hao Lo', 'phone_number': '(872) -810-5588', 'email': 'jiahaolo@u.northwestern.edu', 'local': 'Evanston, Illinois', 'education level': 'Master of Science in Analytics', 'CGPA': 3.83, 'University': 'Northwestern University', 'Education Background': 'Bachelor of Civil Engineering in Offshore Engineering', 'Data Science Background': 'Data Science Department On-Job Training', 'Relevant experience': 'Credit Risk Analysis Industry Practicum'}
dict

{'name': 'Jia-Hao Lo',
 'phone_number': '(872) -810-5588',
 'email': 'jiahaolo@u.northwestern.edu',
 'local': 'Evanston, Illinois',
 'education level': 'Master of Science in Analytics',
 'CGPA': 3.83,
 'University': 'Northwestern University',
 'Education Background': 'Bachelor of Civil Engineering in Offshore Engineering',
 'Data Science Background': 'Data Science Department On-Job Training',
 'Relevant experience': 'Credit Risk Analysis Industry Practicum'}

In [28]:
str(dict)

"{'name': 'Jia-Hao Lo', 'phone_number': '(872) -810-5588', 'email': 'jiahaolo@u.northwestern.edu', 'local': 'Evanston, Illinois', 'education level': 'Master of Science in Analytics', 'CGPA': 3.83, 'University': 'Northwestern University', 'Education Background': 'Bachelor of Civil Engineering in Offshore Engineering', 'Data Science Background': 'Data Science Department On-Job Training', 'Relevant experience': 'Credit Risk Analysis Industry Practicum'}"

In [47]:
from langchain_community.document_loaders import PyPDFLoader

file_path = 'resume/Ang Teik Hun Resume.pdf'
loader = PyPDFLoader(file_path)
documents = loader.load()
def join_page_contents(documents):
    return '\n\n'.join(doc.page_content for doc in documents)

joined_content = join_page_contents(documents)



 
Curriculum Vitae/Resume  
Name: Ang Teik Hun  
 
PROFILE  
A highly motivated and skilled data scientist with a strong background in computer science and statistical analysis. 
Proven experience in using a variety of data mining and machine learning techniques to solve complex problems. 
Proficient in Python, R, and  SQL.  
 
 EXPERIENCE  
Internship as Data Scientist, Petroliam Nasional Berhad  
Group Digital , KL / November 2020  – July 2021  
• Used regular expression and other text pre -processing features in python 
to automate the extraction of information  from corporate business 
database , minimizing data quality issues by 90% . 
• Utilized Object Orient ed Programming (class and subclass) for the 
conversion of data from JSON to panda, performed queries such as 
append, split and find  all to categorize the databa se. 
• Applied techniques like natural processing language processing and text 
mining on corporate business database (Fuzzy mapping), increased 
mapping accuracy

In [45]:
documents

[Document(page_content=' \nCurriculum Vitae/Resume  \nName: Ang Teik Hun  \n \nPROFILE  \nA highly motivated and skilled data scientist with a strong background in computer science and statistical analysis. \nProven experience in using a variety of data mining and machine learning techniques to solve complex problems. \nProficient in Python, R, and  SQL.  \n \n EXPERIENCE  \nInternship as Data Scientist, Petroliam Nasional Berhad  \nGroup Digital , KL / November 2020  – July 2021  \n• Used regular expression and other text pre -processing features in python \nto automate the extraction of information  from corporate business \ndatabase , minimizing data quality issues by 90% . \n• Utilized Object Orient ed Programming (class and subclass) for the \nconversion of data from JSON to panda, performed queries such as \nappend, split and find  all to categorize the databa se. \n• Applied techniques like natural processing language processing and text \nmining on corporate business database (

In [12]:
from langchain_community.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

chat = ChatOpenAI(model='gpt-3.5-turbo-0125',temperature=0.4)
messages = [
    SystemMessage(
        content="""Please act as a hiring manager with 20 years experience. You will be provided a job title and its job description.\n
        [Instruction] 
        1. Determine all of the hiring criteria included in the job description
        2. Skills and qualifications, work experience, Education, Technical certifications, Proficiency with relevant software/tools, Relevant Keywords
        3. Assign weightage to each of the criteria based on its importance in the job position.
        4. Return them in a python list.
        
        [Format of the list]
        [{Criterion : criterion,Weightage : weightage}]"""
    ),
    HumanMessage(
        content=f"""
[Job Title]
Data Scientist

        
[Job Description]

We are looking for experienced Data Scientists to join our team. The ideal candidate should have a minimum of 3 years of work experience in data science projects and possess a strong background in Python, statistical analysis, EDA, Machine Learning, Deep Learning, model deployment and MLOps.


The successful candidate will be responsible for leveraging their expertise in data science to identify the most appropriate models that address business pain points, developing end-to-end solutions for data science projects, and deploying them into production environments The ideal candidate will also possess excellent communication skills to effectively collaborate with stakeholders across the organization.


Key Accountabilities

Working with stakeholders to understand their needs and develop solutions accordingly.
Developing predictive models using machine learning algorithms such as regression analysis, classification trees, neural networks etc.
Analyzing large datasets to identify patterns and trends that can be used to inform decision-making processes.
Utilizing Python programming language for data manipulation and visualization, models development and deployment.
Deploying machine learning and deep learning models into production environments
Developing MLOps pipelines for automated model deployment and monitoring
Documenting all project activities including code development, data analysis results etc.


What are the requirements?

At least bachelor’s degree in Data Science, Computer Science, Engineering, or related field
At least 3 years of work experience in data science projects
Proficiency in Python, data structure and algorithms
Expertise in EDA, Statistical Analysis and Modeling
Expertise in Machine Learning model development & deployment
Expertise in at least one of these areas is preferable; deep learning, text analytic, computer vision, optimization, and simulation.
Experience with Generative AI\Reinforcement learning is preferable.
Strong problem-solving skills & ability to think critically."""
    ),
]
chat.invoke(messages)

AIMessage(content="[{'Criterion': 'Work Experience', 'Weightage': 5}, \n {'Criterion': 'Education', 'Weightage': 4}, \n {'Criterion': 'Skills and Qualifications', 'Weightage': 5}, \n {'Criterion': 'Technical Certifications', 'Weightage': 3}, \n {'Criterion': 'Proficiency with Relevant Software/Tools', 'Weightage': 4}, \n {'Criterion': 'Relevant Keywords', 'Weightage': 4}]")